# Neuroscience using `fastplotlib` and `pynapple`

This notebook will build up a complex visualization using `fastplotlib`, in conjunction with other Python neuroscience packages, to show how `fastplotlib` can be a powerful tool in analysis and visualization of neural data!

In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
import fastplotlib as fpl
import pynapple as nap
import numpy as np
from ipywidgets import IntSlider, Layout, VBox

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.


Available devices:
✅ (default) | NVIDIA GeForce RTX 4060 Laptop GPU | DiscreteGPU | Vulkan | 550.78
❗ | Mesa Intel(R) Arc(tm) Graphics (MTL) | IntegratedGPU | OpenGL | 


## Load the data 

In [3]:
data = nap.load_file("./data.nwb")

In [4]:
data

data
┍━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━┑
│ Keys                  │ Type        │
┝━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━┥
│ position_time_support │ IntervalSet │
│ RoiResponseSeries     │ TsdFrame    │
│ calcium_video         │ TsdTensor   │
│ beh_video             │ TsdTensor   │
│ z                     │ Tsd         │
│ y                     │ Tsd         │
│ x                     │ Tsd         │
│ rz                    │ Tsd         │
│ ry                    │ Tsd         │
│ rx                    │ Tsd         │
┕━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━┙

### Let's view the behavior and calcium data

**Because the recording framerates of the behavior and calcium data are different, we need to synchronize them manually.**

Hopefully, by the end of the summer we will have developed a tool ([`pynaviz`](https://github.com/pynapple-org/pynaviz)) that makes these visualizations and synchronizations even easier :D

In [5]:
# behavior shape
behavior_data = data["beh_video"]
behavior_data.shape

(9045, 204, 256)

In [6]:
# calcium shape
calcium_data = data["calcium_video"]
calcium_data.shape

(17886, 136, 166)

#### Recording Frame Rates:

In [7]:
behavior_fr = 7.5
calcium_fr = 15

In [8]:
# making a timestamp for the behavioral video
t_behavior = np.linspace(0, behavior_data.shape[0] / behavior_fr, behavior_data.shape[0])

In [9]:
t_behavior.shape

(9045,)

In [10]:
t_behavior[-1]

1206.0

#### Use `pynapple` to create an indexable tensor of our behavioral data

In [11]:
tsd_video = nap.TsdTensor(t_behavior, behavior_data.data())

In [12]:
# all of our time points and corresponding behavior frames mapped
tsd_video

Time (s)
--------------  -----------------
0.0             [[35 ... 44] ...]
0.133348076     [[35 ... 44] ...]
0.266696152     [[35 ... 44] ...]
0.400044228     [[35 ... 44] ...]
0.533392304     [[35 ... 44] ...]
0.66674038      [[35 ... 44] ...]
0.800088456     [[35 ... 44] ...]
...
1205.199911544  [[35 ... 44] ...]
1205.33325962   [[35 ... 45] ...]
1205.466607696  [[35 ... 45] ...]
1205.599955772  [[35 ... 44] ...]
1205.733303848  [[35 ... 44] ...]
1205.866651924  [[35 ... 45] ...]
1206.0          [[35 ... 44] ...]
dtype: uint8, shape: (9045, 204, 256)

#### Do the same for our calcium data

In [13]:
# making a timestamp for the calcium video
t_calcium = np.linspace(0, calcium_data.shape[0] / calcium_fr, calcium_data.shape[0])

In [14]:
t_calcium.shape

(17886,)

In [15]:
t_calcium[-1]

1192.4

#### Use `pynapple` to create an indexable tensor of our behavioral data

In [16]:
tsd_calcium = nap.TsdTensor(t_calcium, calcium_data.data())

In [17]:
# all of our time points and corresponding behavior frames mapped
tsd_calcium

Time (s)
--------------  -----------------
0.0             [[25 ... 28] ...]
0.066670394     [[27 ... 28] ...]
0.133340788     [[25 ... 28] ...]
0.200011183     [[26 ... 27] ...]
0.266681577     [[26 ... 28] ...]
0.333351971     [[26 ... 28] ...]
0.400022365     [[26 ... 30] ...]
...
1191.999977635  [[28 ... 32] ...]
1192.066648029  [[25 ... 28] ...]
1192.133318423  [[30 ... 33] ...]
1192.199988817  [[25 ... 27] ...]
1192.266659212  [[28 ... 30] ...]
1192.333329606  [[26 ... 31] ...]
1192.4          [[29 ... 29] ...]
dtype: uint8, shape: (17886, 136, 166)

#### Create a plot for calcium and behavior video

In [18]:
nap_figure = fpl.Figure(shape=(1,2), names=[["raw", "behavior"]])

nap_figure["raw"].add_image(data=tsd_calcium[0], name="raw_frame")
nap_figure["behavior"].add_image(data=tsd_video[0], cmap="gray")

RFBOutputContext()

Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.


<weakproxy at 0x7d5e99d1b560 to ImageGraphic at 0x7d5eaa5e2ed0>

#### Create a slider that updates behavior and calcium data so they are aligned

In [19]:
# This time will be in milliseconds
synced_time = IntSlider(min=0, max=63*19 * 1_000, description="ms", layout=Layout(width="500"))

def update_time(change):
    # get the index of synced slider
    time_ms = change["new"]
    # get the corresponding calcium frame from the pynapple tensor
    frame_raw = tsd_calcium.get(time_ms, time_units="ms")
    # update the data in the plot
    nap_figure["raw"].graphics[0].data = frame_raw
    # get the corresponding behavior frame from the pynapple tensor
    frame_behavior = tsd_video.get(time_ms, time_units="ms")
    # update the data in the plot
    nap_figure["behavior"].graphics[0].data = frame_behavior

synced_time.observe(update_time, "value")

In [21]:
VBox([nap_figure.show(), synced_time])

In [22]:
fig = fpl.Figure()

RFBOutputContext()

In [23]:
data["RoiResponseSeries"].shape

(35772, 105)

In [25]:
data["RoiResponseSeries"]

Time (s)           0        1        2        3        4  ...
-----------  -------  -------  -------  -------  -------  -----
0.0          0        0.43582  2.96331  0        0        ...
0.033333     0        0.43406  2.95294  0        0        ...
0.066667     0        0.43231  2.9426   0        0        ...
0.1          0        0.43057  2.93231  0        0        ...
0.133333     0        0.42883  2.92205  0        0        ...
0.166667     0        0.4271   2.91182  0        0        ...
0.2          0        0.42537  2.90163  0        0        ...
...
1192.166667  2.54202  0.14531  0.44013  0.5681   0.65477  ...
1192.2       2.53029  0.14775  0.43842  0.56657  0.65227  ...
1192.233333  2.51861  0.14962  0.43671  0.56505  0.64979  ...
1192.266667  2.50698  0.15104  0.435    0.56354  0.64731  ...
1192.3       2.49541  0.15209  0.43331  0.56202  0.64485  ...
1192.333333  2.48389  0.15283  0.43162  0.58476  0.64239  ...
1192.366667  2.47242  0.15333  0.42994  0.62802  0.63994  ...
dt